In [84]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import yfinance as yf
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [85]:
# Set the date range for the data collection
end_date = datetime.now()
start_date = end_date - timedelta(days=1500)

# Set the ticker symbol for the company of interest
ticker_symbol = 'AAPL'

# Collect the news articles for the company of interest
url = f'https://news.google.com/rss/search?q={ticker_symbol}&hl=en-US&gl=US&ceid=US:en'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'xml')
news_items = soup.findAll('item')

# Create an empty dataframe to store the data
df = pd.DataFrame(columns=['Date', 'News Headline', 'News Body', 'Sentiment Score', 'Closing Price', 'Next Day Prediction'])

# Collect the stock prices for the company of interest
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Initialize the sentiment analyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[*********************100%***********************]  1 of 1 completed


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lokes\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [86]:
# Loop through the news items and calculate the sentiment score
for news_item in news_items:
    news_date = datetime.strptime(news_item.pubDate.string, '%a, %d %b %Y %H:%M:%S %Z')
    if start_date <= news_date <= end_date:
        news_headline = news_item.title.string
        news_body = news_item.description.string
        sentiment_score = sia.polarity_scores(news_body)['compound']
        try:
            closing_price = stock_data.loc[str(news_date.date())]['Close']
            next_day_prediction = stock_data.loc[str((news_date + timedelta(days=1)).date())]['Close']
        except KeyError:
            # Skip this news item if stock data is not available
            continue
        df = df.append({'Date': news_date, 'News Headline': news_headline, 'News Body': news_body,
                        'Sentiment Score': sentiment_score, 'Closing Price': closing_price, 'Next Day Prediction': next_day_prediction},
                       ignore_index=True)



In [87]:

# Save the data to a CSV file
df.to_csv('news_stock_data.csv', index=False)

In [88]:
import pandas as pd

# Assume that `df` is the DataFrame you want to display as a table
table_html = df.to_html()

# Save the HTML table to a file
with open('table.html', 'w') as f:
    f.write(table_html)
    
from IPython.display import display

# Assume that `df` is the DataFrame you want to display as a table
display(df)


,Date,News Headline,News Body,Sentiment Score,Closing Price,Next Day Prediction
0,2023-04-20 12:00:16,Apple Inc.'s (NASDAQ:AAPL) institutional share...,"<a href=""https://news.google.com/rss/articles/...",0.8625,166.649994,165.001694
1,2023-04-19 10:59:40,"JP Morgan hikes AAPL price target to $190, sti...","<a href=""https://news.google.com/rss/articles/...",0.4404,167.630005,166.649994
2,2023-04-19 11:52:06,Multiple Reasons to Invest in and Hold Apple (...,"<a href=""https://news.google.com/rss/articles/...",0.0000,167.630005,166.649994
3,2023-04-18 17:50:10,Analysts are Upbeat About Apple’s (NASDAQ:AAPL...,"<a href=""https://news.google.com/rss/articles/...",0.0000,166.470001,167.630005
4,2023-04-20 21:36:11,"In certain areas, India could be the new China...","<a href=""https://news.google.com/rss/articles/...",0.2732,166.649994,165.001694
...,...,...,...,...,...,...
66,2023-03-15 07:00:00,Is Apple Inc. (NASDAQ:AAPL) Worth US$153 Based...,"<a href=""https://news.google.com/rss/articles/...",0.3094,152.990005,155.850006
67,2023-01-17 08:00:00,Should Investors Bite Into Apple (AAPL) Stock?...,"<a href=""https://news.google.com/rss/articles/...",0.0000,135.940002,135.210007
68,2022-12-29 08:00:00,Will Apple (AAPL) Stock Bounce Back in 2023? -...,"<a href=""https://news.google.com/rss/articles/...",0.0000,129.610001,129.929993
69,2023-01-31 08:00:00,Apple Stock Forecast: Could AAPL drop 19% from...,"<a href=""https://news.google.com/rss/articles/...",-0.3527,144.289993,145.429993


In [89]:
# create new column 'stock' with values 1 or 0 based on the condition
df['stock'] = df.apply(lambda x: 1 if x['Next Day Prediction'] > x['Closing Price'] else 0, axis=1)

# re-arrange the columns as per your preference
df_new = df[['News Headline', 'Sentiment Score','stock']]
df_new

,News Headline,Sentiment Score,stock
0,Apple Inc.'s (NASDAQ:AAPL) institutional share...,0.8625,0
1,"JP Morgan hikes AAPL price target to $190, sti...",0.4404,0
2,Multiple Reasons to Invest in and Hold Apple (...,0.0000,0
3,Analysts are Upbeat About Apple’s (NASDAQ:AAPL...,0.0000,1
4,"In certain areas, India could be the new China...",0.2732,0
...,...,...,...
66,Is Apple Inc. (NASDAQ:AAPL) Worth US$153 Based...,0.3094,1
67,Should Investors Bite Into Apple (AAPL) Stock?...,0.0000,0
68,Will Apple (AAPL) Stock Bounce Back in 2023? -...,0.0000,1
69,Apple Stock Forecast: Could AAPL drop 19% from...,-0.3527,1


# New code

In [ ]:

#-------------------- Libraries for Email----------------------------------------------------- 
import smtplib
#--------------------Start code for WhatsApp----------------------------------------------------- 
from twilio.rest import Client

def send_whatsapp_notification(sender, to_number, message, account_sid, auth_token):
    client = Client(account_sid, auth_token)
    message=f'Hey there! \nYou have been contacted through IAS group 5.\nYou have received a message from {sender}. \nThe message is{message}'
    message = client.messages.create( 
        body=message,
        from_='whatsapp:+14155238886',
        to=f'whatsapp:{to_number}'
    )

    print(f"WhatsApp notification sent to {to_number}, Message SID: {message.sid}")

# If you wish to change the 'to_number', make sure you send 'join appearance-stove' to '+14155238886'
to_number = '+917506636566'
message = "The message is from Fault Tolerance module. There is a problem with node sid_vm!"
senderName='Sanya Gandhi'

# Account has been already created. So, use the following
account_sid = "ACef0a0280e62b5f9afeb8707d73b3053d"
auth_token = "66ec1a3be77bfc9446a246839b4f7ac0"


# For testing purposes
send_whatsapp_notification(senderName ,to_number, message, account_sid, auth_token)

#------------------------End code for WhatsApp-----------------


#------------------Start code for Email ------------------------
# In order to access mail services, call 'send_mail' function
# Parameters:
# 1. Sender email address
# 2. Receiver email address
# 3. Your app password
## Generate app password by following the steps below:
#       Go to your google account
#       Go to security
#       Go to 2- step verification
#       Create an 'App password' and use the password here
#       Add Subject string
#       Add message body



def send_mail(sender_email , receiver_email, password, sub = "Welcome buddies", body = "Thanks for using our platform, please check the code,\n updated in github"):
    #Broadcasting message to all if it is a list
    if isinstance(receiver_email, list):
        for receiver in receiver_email:
            # Send email to each receiver
            send(sender_email , receiver, password, sub, body)
    else:
        # Send email to single receiver
        send(sender_email , receiver_email, password, sub, body)
         


def send(sender_email ,receiver_email, password, sub = "", body = ""):
    message = f"Subject: {sub} \n\n{body}"
    try:
        server = smtplib.SMTP('smtp.gmail.com')
        #start TLS
        server.starttls()
        try:
            server.login(sender_email, password)
            # print('Login successful to server')
        except Exception as e:
            print(f"Error: {str(e)}")
            print("Failed to login. Please check your credentials and try again.")
            print("Generate less secure key, Check it out https://support.google.com/mail/answer/185833?hl=en")
            return
            
        try:
            # Send the email
            result = server.sendmail(sender_email, receiver_email, message)
            if not result:
                print(f'message send successfully to {receiver_email}')
            else:
                print(f'Failed to send email to {result.keys()}')

        except Exception as e:
            print(f"Error: {str(e)}")
            print("Failed to send email. Please check the email addresses provided and try again.")
            return
        finally:
            #close the connection
            server.quit()
    except Exception as e:
        print(f"Error {str(e)}")
        print("Failed to connect to Smtp. Please check internet connectivity")


        

 